# Import necessary libraries

In [1]:
# For data processing.
import pandas as pd
import numpy as np

# Data Cleaning and Compilation

The first thing we shall do is to clean the data. Our main data will be "series.csv".
We will retrieve information from the other CSVs in order to complete the data in series.csv.
This new data will then be saved in "clean.csv".

In [2]:
# Load genre data from CSV file into a DataFrame.
genres = pd.read_csv('csvs/genres.csv')

# Load network data from CSV file into a DataFrame.
networks = pd.read_csv('csvs/networks.csv')

# Load people data from CSV file into a DataFrame.
people = pd.read_csv('csvs/people.csv')

# Load production company data from CSV file into a DataFrame.
production_companies = pd.read_csv('csvs/production_companies.csv')

# Load series data from CSV file into a DataFrame.
data = pd.read_csv('csvs/series.csv')

In [3]:
# Calculate the number of missing values in each column of the DataFrame.
data.isna().sum()

tmdb_id                        0
name                           0
original_name                  0
keywords                     542
airing_date                   64
poster_img_url                94
background_img_url           332
directors_ids               1350
cast_ids                      40
genres_ids                     0
number_of_seasons              4
number_of_episodes            13
episode_run_time             220
synopsis                      78
popularity                     0
average_rating               387
rating_count                 387
next_episode_to_air         1478
next_episode_air_date       1478
origin_country                46
networks_ids                 205
production_companies_ids     608
dtype: int64

The data has many missing values. However, we will first drop only those in crucial columns.

In [4]:
# Remove rows with missing values in specified columns. 
data.dropna(subset=['average_rating','rating_count','episode_run_time','number_of_episodes','cast_ids','networks_ids'],inplace=True)

# Reset the DataFrame index to be continuous after row removal, and drop the old index.
data.reset_index(drop=True,inplace=True)

# Calculate and display the number of missing values in each column after cleaning to verify no remaining NaNs in crucial columns.
data.isna().sum()

tmdb_id                       0
name                          0
original_name                 0
keywords                    261
airing_date                   0
poster_img_url                4
background_img_url           85
directors_ids               800
cast_ids                      0
genres_ids                    0
number_of_seasons             0
number_of_episodes            0
episode_run_time              0
synopsis                      8
popularity                    0
average_rating                0
rating_count                  0
next_episode_to_air         905
next_episode_air_date       905
origin_country               11
networks_ids                  0
production_companies_ids    300
dtype: int64

Now, let us drop the unnecessary columns.

In [5]:
data.drop(columns=['poster_img_url','background_img_url','next_episode_to_air','next_episode_air_date','origin_country'],inplace=True)
data.shape

(911, 17)

In [6]:
data.head()

,tmdb_id,name,original_name,keywords,airing_date,directors_ids,cast_ids,genres_ids,number_of_seasons,number_of_episodes,episode_run_time,synopsis,popularity,average_rating,rating_count,networks_ids,production_companies_ids
0,99966,All of Us Are Dead,지금 우리 학교는,"high school, bullying, based on comic, horror,...",2022-01-28,1182291,"2125084, 1493898, 2185341, 2074587, 2402531, 1...","10759, 18, 10765",2.0,12.0,65.0,A high school becomes ground zero for a zombie...,398.111,8.421,2750.0,213,"56797, 86981, 113553"
1,93405,Squid Game,오징어 게임,"secret organization, challenge, survival, fict...",2021-09-17,1294471,"73249, 1593672, 3194501, 1557181, 1048070, 191...","10759, 9648, 18",2.0,9.0,54.0,Hundreds of cash-strapped players accept a str...,323.945,7.835,11835.0,213,112647
2,136699,Glitch,글리치,"friendship, investigation, ufo, miniseries, su...",2022-10-07,1293597,"2067860, 1523043, 1418580, 2239782, 150697, 17...","18, 35, 9648, 10765",1.0,10.0,54.0,A young woman joins forces with a UFO enthusia...,223.442,7.674,43.0,213,133335
3,197067,Extraordinary Attorney Woo,이상한 변호사 우영우,"court case, court, autism, lawyer, courtroom, ...",2022-06-29,1920791,"1134684, 1565392, 1648169, 1572347, 2720442, 5...","18, 35",1.0,16.0,70.0,Brilliant attorney Woo Young-woo tackles chall...,147.054,8.310,381.0,"213, 5841","87422, 165813"
4,129473,Young Lady and Gentleman,신사와 아가씨,"tutor, family, single father, healing, rich ma...",2021-09-25,NaN,"1253552, 2752543, 1568455, 1254981, 1487437, 1...","35, 18, 10751",1.0,52.0,70.0,Lee Young Kook is a widower with three childre...,128.825,8.125,8.0,342,NaN


Let us first convert all the comma-separated fields into lists. Then, we shall convert the respective IDs into their corresponding strings.
We shall also add a cast popularity field to be used later.

In addition, since they are no longer in ID form, we shall rename the columns.

In [7]:
# Split string entries in specified columns by ', ' and convert to lists for easier manipulation.
for c in ['keywords','directors_ids','cast_ids','genres_ids','networks_ids','production_companies_ids']:
    data[c] = data[c].str.split(', ')
data.head()

,tmdb_id,name,original_name,keywords,airing_date,directors_ids,cast_ids,genres_ids,number_of_seasons,number_of_episodes,episode_run_time,synopsis,popularity,average_rating,rating_count,networks_ids,production_companies_ids
0,99966,All of Us Are Dead,지금 우리 학교는,"[high school, bullying, based on comic, horror...",2022-01-28,[1182291],"[2125084, 1493898, 2185341, 2074587, 2402531, ...","[10759, 18, 10765]",2.0,12.0,65.0,A high school becomes ground zero for a zombie...,398.111,8.421,2750.0,[213],"[56797, 86981, 113553]"
1,93405,Squid Game,오징어 게임,"[secret organization, challenge, survival, fic...",2021-09-17,[1294471],"[73249, 1593672, 3194501, 1557181, 1048070, 19...","[10759, 9648, 18]",2.0,9.0,54.0,Hundreds of cash-strapped players accept a str...,323.945,7.835,11835.0,[213],[112647]
2,136699,Glitch,글리치,"[friendship, investigation, ufo, miniseries, s...",2022-10-07,[1293597],"[2067860, 1523043, 1418580, 2239782, 150697, 1...","[18, 35, 9648, 10765]",1.0,10.0,54.0,A young woman joins forces with a UFO enthusia...,223.442,7.674,43.0,[213],[133335]
3,197067,Extraordinary Attorney Woo,이상한 변호사 우영우,"[court case, court, autism, lawyer, courtroom,...",2022-06-29,[1920791],"[1134684, 1565392, 1648169, 1572347, 2720442, ...","[18, 35]",1.0,16.0,70.0,Brilliant attorney Woo Young-woo tackles chall...,147.054,8.310,381.0,"[213, 5841]","[87422, 165813]"
4,129473,Young Lady and Gentleman,신사와 아가씨,"[tutor, family, single father, healing, rich m...",2021-09-25,NaN,"[1253552, 2752543, 1568455, 1254981, 1487437, ...","[35, 18, 10751]",1.0,52.0,70.0,Lee Young Kook is a widower with three childre...,128.825,8.125,8.0,[342],NaN


In [8]:
def find_name(id, col):
    """
    Retrieve the name associated with a given TMDB ID from different dataframes based on the column specified.

    Parameters:
    - id (int): The TMDB ID for which the name is to be retrieved.
    - col (str): Specifies the dataframe column that determines which dataframe to search. 
                 The columns include 'directors_ids', 'cast_ids', 'genres_ids', 
                 'networks_ids', and 'production_companies_ids'.

    Returns:
    - str: The name associated with the given ID from the specified dataframe.

    The function checks the column specified and queries the appropriate dataframe to fetch the name.
    It handles different categories of data: directors, cast members, genres, networks, and production companies.
    """
    if col=='directors_ids' or col=='cast_ids':
        return people.loc[people['tmdb_id']==id].name.values[0]
    elif col=='genres_ids':
        return genres.loc[genres['tmdb_id']==id].name.values[0]
    elif col=='networks_ids':
        return networks.loc[networks['tmdb_id']==id].name.values[0]
    elif col=='production_companies_ids':
        return production_companies.loc[production_companies['tmdb_id']==id].name.values[0]

In [9]:
# Initialize 'cast_popularity' to zero for all rows in the DataFrame.
data['cast_popularity'] = 0

# Iterate over specified columns that contain lists of TMDB IDs.
for c in ['directors_ids','cast_ids','genres_ids','networks_ids','production_companies_ids']:
    # Iterate through each row in the DataFrame.
    for i, row in data.iterrows(): 
        # Skip processing if the cell does not contain a list.
        if type(row[c]) is not list:
                continue
        # Initialize a temporary list to hold names corresponding to the IDs.        
        intemp = []
        # Convert each ID in the current cell to an integer and find the corresponding name using the find_name function.
        for f in row[c]:
            intemp.append(find_name(int(f), c))
        # If processing "cast IDs", accumulate popularity scores for each cast member.    
        if c=='cast_ids':
            for f in row[c]:
                # Add the 'popularity' value from the 'people' DataFrame to the 'cast_popularity' for the current row.
                data['cast_popularity'].loc[i] += people.loc[people.tmdb_id==int(f)].popularity.values[0]
        # Update the current cell with the list of names retrieved.        
        data[c].loc[i] = intemp

C:\Users\glenn\AppData\Local\Temp\ipykernel_82116\3576395097.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '8.427' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['cast_popularity'].loc[i] += people.loc[people.tmdb_id==int(f)].popularity.values[0]


In [10]:
# Rename the columns, since they are no longer in ID form.
data.rename(columns={'directors_ids':'directors','cast_ids':'cast','genres_ids':'genres','networks_ids':'networks','production_companies_ids':'production_companies'},inplace=True)
data.head()

,tmdb_id,name,original_name,keywords,airing_date,directors,cast,genres,number_of_seasons,number_of_episodes,episode_run_time,synopsis,popularity,average_rating,rating_count,networks,production_companies,cast_popularity
0,99966,All of Us Are Dead,지금 우리 학교는,"[high school, bullying, based on comic, horror...",2022-01-28,[JQ Lee],"[Park Ji-hu, Yoon Chan-young, Cho Yi-hyun, Lom...","[Action & Adventure, Drama, Sci-Fi & Fantasy]",2.0,12.0,65.0,A high school becomes ground zero for a zombie...,398.111,8.421,2750.0,[Netflix],"[Kim Jong-hak Production, SLL, Film Monster]",82.507
1,93405,Squid Game,오징어 게임,"[secret organization, challenge, survival, fic...",2021-09-17,[Hwang Dong-hyuk],"[Lee Jung-jae, Park Hae-soo, Jung Ho-yeon, Wi ...","[Action & Adventure, Mystery, Drama]",2.0,9.0,54.0,Hundreds of cash-strapped players accept a str...,323.945,7.835,11835.0,[Netflix],[Siren Pictures],68.428
2,136699,Glitch,글리치,"[friendship, investigation, ufo, miniseries, s...",2022-10-07,[Roh Deok],"[Jeon Yeo-been, Nana, Lee Dong-hwi, Ryu Kyung-...","[Drama, Comedy, Mystery, Sci-Fi & Fantasy]",1.0,10.0,54.0,A young woman joins forces with a UFO enthusia...,223.442,7.674,43.0,[Netflix],[Studio 329],81.095
3,197067,Extraordinary Attorney Woo,이상한 변호사 우영우,"[court case, court, autism, lawyer, courtroom,...",2022-06-29,[Yoo In-sik],"[Park Eun-bin, Kang Tae-oh, Kang Ki-young, Jeo...","[Drama, Comedy]",1.0,16.0,70.0,Brilliant attorney Woo Young-woo tackles chall...,147.054,8.310,381.0,"[Netflix, ENA]","[AStory, KT Studio Genie]",131.091
4,129473,Young Lady and Gentleman,신사와 아가씨,"[tutor, family, single father, healing, rich m...",2021-09-25,NaN,"[Ji Hyun-woo, Lee Se-hee, Park Ha-na, Oh Hyun-...","[Comedy, Drama, Family]",1.0,52.0,70.0,Lee Young Kook is a widower with three childre...,128.825,8.125,8.0,[KBS2],NaN,71.909


We shall now finish up cleaning by joining list items by semicolons so that they can be easily separated in the next notebook.
Then, we will write the new data to "clean.csv".

In [11]:
for c in ['keywords','directors','cast','genres','networks','production_companies']:
    data[c] = data[c].str.join(';')
data

,tmdb_id,name,original_name,keywords,airing_date,directors,cast,genres,number_of_seasons,number_of_episodes,episode_run_time,synopsis,popularity,average_rating,rating_count,networks,production_companies,cast_popularity
0,99966,All of Us Are Dead,지금 우리 학교는,high school;bullying;based on comic;horror;com...,2022-01-28,JQ Lee,Park Ji-hu;Yoon Chan-young;Cho Yi-hyun;Lomon;Y...,Action & Adventure;Drama;Sci-Fi & Fantasy,2.0,12.0,65.0,A high school becomes ground zero for a zombie...,398.111,8.421,2750.0,Netflix,Kim Jong-hak Production;SLL;Film Monster,82.507
1,93405,Squid Game,오징어 게임,secret organization;challenge;survival;fiction...,2021-09-17,Hwang Dong-hyuk,Lee Jung-jae;Park Hae-soo;Jung Ho-yeon;Wi Ha-j...,Action & Adventure;Mystery;Drama,2.0,9.0,54.0,Hundreds of cash-strapped players accept a str...,323.945,7.835,11835.0,Netflix,Siren Pictures,68.428
2,136699,Glitch,글리치,friendship;investigation;ufo;miniseries;suspense,2022-10-07,Roh Deok,Jeon Yeo-been;Nana;Lee Dong-hwi;Ryu Kyung-soo;...,Drama;Comedy;Mystery;Sci-Fi & Fantasy,1.0,10.0,54.0,A young woman joins forces with a UFO enthusia...,223.442,7.674,43.0,Netflix,Studio 329,81.095
3,197067,Extraordinary Attorney Woo,이상한 변호사 우영우,court case;court;autism;lawyer;courtroom;law f...,2022-06-29,Yoo In-sik,Park Eun-bin;Kang Tae-oh;Kang Ki-young;Jeon Ba...,Drama;Comedy,1.0,16.0,70.0,Brilliant attorney Woo Young-woo tackles chall...,147.054,8.310,381.0,Netflix;ENA,AStory;KT Studio Genie,131.091
4,129473,Young Lady and Gentleman,신사와 아가씨,tutor;family;single father;healing;rich man po...,2021-09-25,NaN,Ji Hyun-woo;Lee Se-hee;Park Ha-na;Oh Hyun-kyun...,Comedy;Drama;Family,1.0,52.0,70.0,Lee Young Kook is a widower with three childre...,128.825,8.125,8.0,KBS2,NaN,71.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,112600,Discipline Z: Vampire,디시플린 Z: 벙피르,NaN,2020-10-30,NaN,Oh Dong Joon;Kim Chan Kyu;Ahn Dae Kyum;Kim Yoo...,Drama;Mystery,1.0,6.0,18.0,A YouTube streamer who is a big fan of mysteri...,0.676,3.000,2.0,YouTube,NaN,3.000
907,135706,A First Love Story,퍼스트러브스토리,miniseries;lgbt;military enlistment,2021-08-27,NaN,Kim Hyeong Won;Jung Jae Woon,Drama,1.0,2.0,8.0,"""I'm going to the army.""\n\nMin Kyu was surpri...",0.600,9.000,1.0,YouTube,STRONGBERRY,1.200
908,123392,Nara's Marvelous Days,오늘도 평화로운 중고'나라',NaN,2021-01-21,NaN,Hana;Kang Tae Joo;Lee Ka-eun,Comedy;Drama,1.0,6.0,15.0,A woman begins to think of herself as damaged ...,0.649,7.000,1.0,NAVER TV,NaN,2.228
909,77621,Seventeen,열일곱,friendship;romance;school,2017-04-27,NaN,Yoo Hye-In;Kim Do-wan;Kang Yul;Shin Joohyup;Ki...,Kids;Drama,1.0,8.0,5.0,"At seventeen, we were all newbies at love. The...",1.128,4.000,1.0,NAVER TV,Playlist,8.335


In [12]:
# Write the new data to "clean.csv".
f = open("csvs/clean.csv", "w+")
data.to_csv("csvs/clean.csv")
f.close()